In [1]:
import pandas as pd
import scipy.linalg as la
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

In [6]:
file_path = "/Users/jacobwamon/Documents/synesthesia project/individualCSVs/P002V.csv"
file_path2 = "/Users/jacobwamon/Documents/synesthesia project/individualCSVs/P002A.csv"
df = pd.read_csv(file_path)
df2 = pd.read_csv(file_path2)


In [9]:
def rgb_diff(group):
    rgb_values = group[['r', 'g', 'b']].values
    #print(rgb_values)
    rgb1, rgb2, rgb3 = rgb_values[0], rgb_values[1], rgb_values[2]
    rgb_diff = abs(rgb1 - rgb2) + abs(rgb2 - rgb3) + abs(rgb3 - rgb1)

    #returns a new dataset with the average difference in RGB values for each stimulus
    return pd.Series(rgb_diff, index=['v_diff_r', 'v_diff_g', 'v_diff_b'])

def finalize_score(result):
    
    nan_counts = result[['diff_r' , 'diff_g' , 'diff_b']].isna().any(axis=1).sum()
    #N = len(result) - nan_counts
    result = result.dropna(subset=['diff_r', 'diff_g', 'diff_b'])
    result_norm = result[['diff_r', 'diff_g', 'diff_b']] / 255
    
    N = len(result)

    test_score_rgb = result_norm.sum() / N
    final_test_score = test_score_rgb.sum()
    
    return final_test_score




In [10]:
# these lines of code calculates the RGB differences for each stimuli across the two tests

 
# these are the two tests.
# Group by 'stimulus' and apply the rgb_diff function

# V denotes the visual condition
v_result = df.groupby('stimulus').apply(rgb_diff).reset_index()
# A denotes the auditory condition
a_result = df2.groupby('stimulus').apply(rgb_diff).reset_index()



# Here we just rename each column in the auditory test because the rgb_diff function returns labels specified for V condition.
# Renaming will be important when we merge the datasets later on
new_column_names = {'v_diff_r': 'a_diff_r', 'v_diff_g': 'a_diff_g', 'v_diff_b': 'a_diff_b'}
new_column_names = {'v_diff_r': 'a_diff_r', 'v_diff_g': 'a_diff_g', 'v_diff_b': 'a_diff_b'}
a_result.rename(columns=new_column_names, inplace=True)


# merges the two dataframes from visual and auditory tests
total_score = v_result.merge(a_result, on='stimulus')

# the dataset changed shape from (4, 35) to (7, 35)
print(f"The dataset changed shape from {v_result.shape} to {total_score.shape}")


# now we want to rename the columns in A condition and V condition. We do this just to utilize the same functions
# to guarantee consistency when handling the data 
# now that total score dataset has been created, we can perform some simple arithmetics to get the total 
# difference in RGB values:
total_score['diff_r'] = abs(total_score['v_diff_r'] - total_score['a_diff_r'])
total_score['diff_g'] = abs(total_score['v_diff_g'] - total_score['a_diff_g'])
total_score['diff_b'] = abs(total_score['v_diff_b'] - total_score['a_diff_b'])
total_score

# Visual condition
v_new_column_names = {'v_diff_r': 'diff_r', 'v_diff_g': 'diff_g', 'v_diff_b': 'diff_b'}
v_result.rename(columns=v_new_column_names, inplace=True)
# Auditory condition
a_new_column_names = {'a_diff_r': 'diff_r', 'a_diff_g': 'diff_g', 'a_diff_b': 'diff_b'}
a_result.rename(columns=a_new_column_names, inplace=True)





total_score['v_diff_r']


# here's where the score is normalized and divided by N:
#score = finalize_score(v_result)
#print(f"the final score of the participant is: {twin_score}")

#creates the dataset for twin score
twin_dataset = total_score[['diff_r', 'diff_g', 'diff_b']]

#normalizes the value 
twin_score = finalize_score(total_score)
v_score = finalize_score(v_result)
a_score = finalize_score(a_result)
print(f"The visual score of the participant is: {v_score}")
print(f"The auditory score of the participant is: {a_score}")
print(f"the twin score of the participant is: {twin_score}")
                          

The dataset changed shape from (36, 4) to (36, 7)
The visual score of the participant is: 3.127668845315904
The auditory score of the participant is: 2.9135076252723313
the twin score of the participant is: 1.8233115468409586
